In [1]:
# installing open java development kit
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# getting spark package
!wget -q https://archive.apache.org/dist/spark/spark-3.2.0/spark-3.2.0-bin-hadoop3.2.tgz

# unzipping
!tar xf spark-3.2.0-bin-hadoop3.2.tgz

# Installing findspark
!pip install -q findspark

In [2]:
# importing OS
import os

# setting java environment
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

# setting spark environment
os.environ["SPARK_HOME"] = "spark-3.2.0-bin-hadoop3.2"

In [3]:
# importing spark
import findspark

# initializing spark
findspark.init()

In [4]:
# importing spark session
from pyspark.sql import SparkSession

# creating spark session
spark = SparkSession.builder.master("local[*]").getOrCreate()

# getting sparkcontext from spark
sc = spark.sparkContext

# print
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [5]:
# defining the file path
# renamed the file from graph data.csv to graph_data.csv
path = "/content/graph_data.csv"

In [6]:
# reading the data as an rdd
rdd = sc.textFile(path)

In [7]:
# splitting the data inside the rdd
rdd = rdd.map(lambda x:x.split(","))

In [8]:
# converting the values inside the rdd into integers
rdd = rdd.map(lambda x: [int(i) for i in x])

In [9]:
# Creating a tuple of (index + 1, list of neighbor node IDs) for each row in the RDD
index_and_neighbors = rdd.zipWithIndex().map(lambda x: (x[1] + 1, x[0]))

In [10]:
# defining a function to extract neighbor node IDs from a every node(row)
def extract_neighbors(row):
    # defining the components of the row
    key,value = row

    # intializing an empty list for neighbors
    neighbors = []

    # looping through the value part of each row
    for index, val in enumerate(value):

         # Checking if the value is equal to 1 (indicating a neighbor)
        if val == 1:

            # appending the neighbor's index + 1 to the list
            neighbors.append(index + 1)

    # returns a tuple with the node and its neighbors
    return (key, neighbors)

In [11]:
# applying the predefined function to the index_and_neighbors
adjacency_list = index_and_neighbors.map(extract_neighbors)

In [12]:
# print
adjacency_list.collect()

[(1, [2, 4, 5, 7, 8, 9, 12, 14, 15, 17, 18, 20, 22, 24, 25]),
 (2, [1, 4, 7, 9, 12, 15, 17, 19, 22, 24]),
 (3, [2, 4, 5, 9, 14, 15, 18, 22, 23, 25]),
 (4, [2, 3, 7, 11, 12, 14, 17, 18, 21, 23, 24]),
 (5, [3, 4, 6, 7, 8, 12, 14, 17, 18, 19, 24, 25]),
 (6, [1, 3, 4, 7, 9, 11, 12, 14, 17, 18, 21, 22, 23]),
 (7, [4, 6, 8, 9, 11, 12, 15, 18, 19, 21, 22, 25]),
 (8, [2, 4, 10, 11, 12, 13, 14, 15, 19, 22, 23, 24]),
 (9, [1, 4, 5, 8, 10, 11, 14, 16, 20, 22, 23]),
 (10, [1, 2, 4, 5, 9, 11, 13, 16, 17, 19, 22, 24, 25]),
 (11, [1, 2, 4, 7, 9, 12, 17, 18, 19, 22, 23, 24]),
 (12, [3, 4, 5, 7, 10, 11, 14, 15, 21, 22, 23]),
 (13, [1, 2, 4, 10, 11, 12, 14, 16, 17, 19, 24]),
 (14, [2, 5, 6, 8, 9, 13, 18, 19, 21, 22, 24]),
 (15, [2, 3, 4, 5, 11, 16, 18, 20, 21, 23, 25]),
 (16, [1, 2, 7, 9, 11, 12, 13, 19, 21, 22, 24]),
 (17, [2, 4, 8, 9, 14, 15, 16, 22, 24]),
 (18, [3, 4, 9, 11, 12, 14, 16, 17, 22, 25]),
 (19, [2, 11, 14, 16, 17, 22, 24]),
 (20, [2, 3, 7, 10, 15, 16, 17, 19, 22, 25]),
 (21, [3, 4, 6, 7, 

1. Find all self-loops (i.e. edge between a node onto itself)

In [13]:
# filtering the nodes if the node itself is present in the list of neighbors (self_looping)
self_loops = adjacency_list.filter(lambda x: x[0] in x[1]).collect()

In [14]:
# print
print("Answer, Number of nodes with Self-loops:", self_loops)


Answer, Number of nodes with Self-loops: []


2. Node with the largest out-degree

In [15]:
# calculating the length of each node's neighbor list
out_degree = adjacency_list.map(lambda x: (x[0], len(x[1])))

In [16]:
# computing the node with highest number of out_degree
out_degree = out_degree.reduce(lambda x, y: x if x[1] >= y[1] else y)

In [17]:
# print
print("Answer, Node with the largest out-degree:", out_degree[0])

Answer, Node with the largest out-degree: 1


3. Node with the larges in-degree

In [18]:
# creating a new RDD with (neighbor, 1) for each neighbor in the adjacency list
neighbor_counts = adjacency_list.flatMap(lambda x: [(neighbor, 1) for neighbor in x[1]])

In [19]:
# Using reduceByKey to count the in-degrees for each node
in_degree = neighbor_counts.reduceByKey(lambda x, y: x + y)

In [20]:
# finding the node with the largest in-degree
largest = in_degree.max(key=lambda x: x[1])

In [21]:
# print
print("Answer, Node with the largest in-degree:", largest[0])

Answer, Node with the largest in-degree: 4


4. Find the distribution of vertices in-degrees

In [22]:
# creating a new RDD with (neighbor, 1) for each neighbor in the adjacency list
neighbor_counts = adjacency_list.flatMap(lambda x: [(neighbor, 1) for neighbor in x[1]])

In [23]:
# Using countBykey to count the in-degrees for each node
in_degree_distribution = neighbor_counts.countByKey()

In [24]:
# Sorting the dictionary items based on keys in ascending order
in_degree_distribution = dict(sorted(in_degree_distribution.items()))

In [25]:
# Print the sorted in-degree distribution
print("the distribution of vertices in-degrees:")
for node, count in in_degree_distribution.items():
    print(f"Node {node}: {count}")

the distribution of vertices in-degrees:
Node 1: 8
Node 2: 14
Node 3: 10
Node 4: 20
Node 5: 8
Node 6: 4
Node 7: 11
Node 8: 7
Node 9: 15
Node 10: 6
Node 11: 14
Node 12: 14
Node 13: 6
Node 14: 15
Node 15: 11
Node 16: 11
Node 17: 13
Node 18: 10
Node 19: 12
Node 20: 5
Node 21: 9
Node 22: 18
Node 23: 11
Node 24: 16
Node 25: 9


5. Find a path between node 1 to node 9 [output: a list of nodes that connects 1 and 9]

In [28]:
# defining a function to find a path
def finding_path_breadth(rdd_of_nodes, start_node, target_node):

    # creating a set to keep track of visited codes
    visited = set()
    queue = [(start_node, [start_node])]

    # while loop so that the loop continues as long as there are nodes in the queue to explore
    while queue:

        # popping the first node and it's path
        (node, path) = queue.pop(0)

        # If the current node matches the target node, return the path
        if node == target_node:
            return path

        # If the current node hasn't been visited, mark it as visited by adding it to the set
        if node not in visited:
            visited.add(node)

            # finding the neighbors of the current node by filtering the graph RDD
            neighbors = rdd_of_nodes.filter(lambda x: x[0] == node).first()[1]

            # for each neighbor, if it hasn't been visited, add it to the queue with an extended path
            for neighbor in neighbors:
                if neighbor not in visited:
                    queue.append((neighbor, path + [neighbor]))

    # If no path is found, return None
    return None

In [29]:
# calling the predefined function
path = finding_path_breadth(adjacency_list, 1, 9)

if path:
    print("Answer, Path from 1 to 9:", path)
else:
    print("Answer, No path found from 1 to 9.")

Answer, Path from 1 to 9: [1, 9]
